<a href="https://colab.research.google.com/github/RADson2005official/Tax-LLaMA-Ind/blob/colab-notebook/colab_training_easy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tax-LLaMA-Ind: EASY Setup with Google Colab Secrets

## STEP 1: Add Your Hugging Face Token to Colab Secrets
1. Click the **🔑 key icon** in the left sidebar ("Secrets")
2. Click **"+ Add new secret"**
3. Name: `HF_TOKEN`
4. Value: Paste your Hugging Face token from https://huggingface.co/settings/tokens
5. Toggle **"Notebook access"** to ON

## STEP 2: Run All Cells
Click `Runtime` → `Run all`

In [1]:
# Install Dependencies
!pip install -q -U torch transformers peft bitsandbytes trl datasets scipy accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90

In [2]:
# Import libraries and authenticate with Colab Secrets
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
from datasets import load_dataset
from huggingface_hub import login
from google.colab import userdata

# Get token from Colab Secrets
HF_TOKEN = userdata.get('Llama-3.2-8B-Instruct')
login(token=HF_TOKEN)
print("✅ Successfully authenticated with Hugging Face!")

✅ Successfully authenticated with Hugging Face!


In [3]:
# Configuration
# Using Unsloth model which is public and pre-quantized (No access request needed)
MODEL_NAME = "unsloth/llama-3-8b-Instruct-bnb-4bit"
NEW_MODEL_NAME = "Tax-LLaMA-Ind-v1"

# QLoRA Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print(f"Loading model {MODEL_NAME}...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("✅ Model loaded successfully!")

Loading model unsloth/llama-3-8b-Instruct-bnb-4bit...


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

✅ Model loaded successfully!


In [4]:
# Upload your instruction_tuning.jsonl file first!
# You can drag and drop it to the file browser on the left

dataset = load_dataset("json", data_files="/content/instruction_tuning.jsonl", split="train")

def format_instruction(sample):
    return f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{sample['instruction']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{sample['output']}<|eot_id|>"""

dataset = dataset.map(lambda x: {"text": format_instruction(x)})
print(f"✅ Dataset loaded: {len(dataset)} examples")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

✅ Dataset loaded: 5 examples


In [5]:
# LoRA Configuration
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Training Arguments
training_arguments = TrainingArguments(
    output_dir="./tax-llama-checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments
)

print("✅ Trainer configured successfully!")

Adding EOS to train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

✅ Trainer configured successfully!


In [6]:
# Start Training (this will take 1-2 hours)
print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

🚀 Starting training...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nagosejayraj2005 (nagosejayraj2005-sanjivani-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


✅ Training complete!


In [7]:
# Save the model
trainer.model.save_pretrained(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)
print(f"✅ Model saved to {NEW_MODEL_NAME}")

# Optional: Upload to Hugging Face Hub
# trainer.model.push_to_hub(NEW_MODEL_NAME)
# tokenizer.push_to_hub(NEW_MODEL_NAME)

✅ Model saved to Tax-LLaMA-Ind-v1
